# Data Ingestion: Motion Sense

## Initial Setup

In [3]:
#Import python modules
import boto3
import configparser
import json
import pandas as pd
import random
import time


In [4]:
#Load configurations 
#Ask Riu for access key information and update config file before running
%run load_config.py

In [5]:
#Define firehose object
firehose = boto3.client(
    'firehose',
    region_name = "us-east-2",
    aws_access_key_id = access_key_id,
    aws_secret_access_key = secret_access_key)

## Prepare Data for Streaming

In [20]:
#Read data
r_path = "../../../MotionSense/Data/"
csv_name = "df_all_data.csv"

df = pd.read_csv(r_path + csv_name, low_memory = False)

,tick_num,attitude.roll,attitude.pitch,attitude.yaw,gravity.x,gravity.y,gravity.z,rotationRate.x,rotationRate.y,rotationRate.z,...,time_since_start,time_series_data,attitude,gravity,rotationRate,userAcceleration,weight,height,age,gender
1,1,1.527992,-0.716987,0.677762,0.753099,0.657116,-0.032255,0.842032,0.424446,0.643574,...,0.02,2022-01-01 00:00:00.020,1.818843,1.000000,1.141648,0.250235,102,188,46,1
2,2,1.527765,-0.706999,0.670951,0.759611,0.649555,-0.032707,-0.138143,-0.040741,0.343563,...,0.04,2022-01-01 00:00:00.040,1.812205,1.000000,0.372530,0.215450,102,188,46,1
3,3,1.516768,-0.704678,0.675735,0.760709,0.647788,-0.041140,-0.025005,-1.048717,0.035860,...,0.06,2022-01-01 00:00:00.060,1.803822,1.000000,1.049628,0.166728,102,188,46,1
4,4,1.493941,-0.703918,0.672994,0.760062,0.647210,-0.058530,0.114253,-0.912890,0.047341,...,0.08,2022-01-01 00:00:00.080,1.783334,1.000000,0.921229,0.408720,102,188,46,1
5,5,1.476302,-0.700807,0.669443,0.760912,0.644835,-0.072116,0.187742,-0.763656,0.226057,...,0.10,2022-01-01 00:00:00.100,1.765999,1.000000,0.818242,0.223044,102,188,46,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,1.732957,-0.884972,1.845189,0.625003,0.773897,0.102249,-0.879328,-0.074183,-0.488383,...,1.90,2022-01-01 00:00:01.900,2.681611,1.000000,1.008583,0.167286,102,188,46,1
96,96,1.747643,-0.892408,1.866475,0.617752,0.778585,0.110400,-0.473213,-0.115180,-0.444901,...,1.92,2022-01-01 00:00:01.920,2.708205,1.000000,0.659647,0.176391,102,188,46,1
97,97,1.669818,-0.839965,1.816442,0.664219,0.744620,0.065988,-0.372399,-0.314253,-0.280073,...,1.94,2022-01-01 00:00:01.940,2.606395,1.000000,0.562029,0.270372,102,188,46,1
98,98,1.668462,-0.843037,1.827050,0.662028,0.746667,0.064863,-0.308166,-0.581435,-0.120117,...,1.96,2022-01-01 00:00:01.960,2.613922,1.000000,0.668925,0.303608,102,188,46,1


In [21]:
#Column names
column_names = ["tick_num","attitude.roll","attitude.pitch","attitude.yaw","gravity.x","gravity.y","gravity.z","rotationRate.x","rotationRate.y","rotationRate.z","userAcceleration.x","userAcceleration.y","userAcceleration.z","test_type","subject_id","test_trial_number","time_since_start","time_series_data","attitude","gravity","rotationRate","userAcceleration","weight","height","age","gender"]

#Convert specified columns from int64 to string
df["tick_num"] = df["tick_num"].astype(str)
df["subject_id"] = df["subject_id"].astype(str)
df["test_trial_number"] = df["test_trial_number"].astype(str)
df["weight"] = df["weight"].astype(str)
df["height"] = df["height"].astype(str)
df["age"] = df["age"].astype(str)
df["gender"] = df["gender"].astype(str)

In [ ]:
#Keep only test data for simulating streaming
#df = (filter data to only keep test data)

## Simulate Streaming

In [22]:
#Define number of rows in data
nrow = df.shape[0]

In [23]:
#Simulate data streaming
for i in range(nrow):
    response = firehose.put_record(
        DeliveryStreamName = DeliveryStreamName,
        Record = {
            "Data": json.dumps(
                {
                    f"{column_names[0]}": df.at[df.index[i], column_names[0]],
                    f"{column_names[1]}": df.at[df.index[i], column_names[1]],
                    f"{column_names[2]}": df.at[df.index[i], column_names[2]],
                    f"{column_names[3]}": df.at[df.index[i], column_names[3]],
                    f"{column_names[4]}": df.at[df.index[i], column_names[4]],
                    f"{column_names[5]}": df.at[df.index[i], column_names[5]],
                    f"{column_names[6]}": df.at[df.index[i], column_names[6]],
                    f"{column_names[7]}": df.at[df.index[i], column_names[7]],
                    f"{column_names[8]}": df.at[df.index[i], column_names[8]],
                    f"{column_names[9]}": df.at[df.index[i], column_names[9]],
                    f"{column_names[10]}": df.at[df.index[i], column_names[10]],
                    f"{column_names[11]}": df.at[df.index[i], column_names[11]],
                    f"{column_names[12]}": df.at[df.index[i], column_names[12]],
                    f"{column_names[13]}": df.at[df.index[i], column_names[13]],
                    f"{column_names[14]}": df.at[df.index[i], column_names[14]],
                    f"{column_names[15]}": df.at[df.index[i], column_names[15]],
                    f"{column_names[16]}": df.at[df.index[i], column_names[16]],
                    f"{column_names[17]}": df.at[df.index[i], column_names[17]],
                    f"{column_names[18]}": df.at[df.index[i], column_names[18]],
                    f"{column_names[19]}": df.at[df.index[i], column_names[19]],
                    f"{column_names[20]}": df.at[df.index[i], column_names[20]],
                    f"{column_names[21]}": df.at[df.index[i], column_names[21]],
                    f"{column_names[22]}": df.at[df.index[i], column_names[22]],
                    f"{column_names[23]}": df.at[df.index[i], column_names[23]],
                    f"{column_names[24]}": df.at[df.index[i], column_names[24]],
                    f"{column_names[25]}": df.at[df.index[i], column_names[25]]
                }
            )
        }
    )
    time.sleep(round(random.uniform(0.01, 0.15), 3))